课程 ：人工智能原理  
学号：2019211955 （大四）  
姓名：陈可淇  
教师： 谭咏梅  
题目：对于n*n数码问题的任意一个排列是否有解？有解的条件是什么？  
~~~~


老师你好，我认为N数码问题是否有解取决于这个排列的逆序对数的奇偶性，逆序数对奇偶性要和目标状态奇偶性( 0 )相同，否则无解。关于n数码问题的详细分析和代码实现可见附件中的.ipynb文件（需要用jupyter notebook打开）。
不过我认为课堂上的讲解稍微缺乏代码实践，我建议将n数码问题同具体代码实现相结合，这是n数码问题的OJ题目网址 https://leetcode.cn/problems/sliding-puzzle/ ，附件中包含我对N数码问题的分析和可通过的代码。

对于n*n数码问题的任意一个排列，不一定都有解。是否有解取决于这个排列的逆序对数的奇偶性。  


对于一个n*n数码问题的排列，将所有大于该数码的数（不包括空格）统计出来，称为该数码的逆序对数。  
例如，对于排列 2 4 3 1  

2的逆序对数为0，4的逆序对数为1（因为3比4小），3的逆序对数为1（因为1比3小），1的逆序对数为0。
因此，该排列的逆序对数为2。  

如果一个排列的逆序对数为偶数，那么它是有解的，否则无解。这是因为每一步移动都会改变逆序对数的奇偶性，而最终的目标状态的逆序对数是0，所以如果初始状态的逆序对数是偶数，那么可以通过一系列合法移动到达目标状态，否则无法到达目标状态。
  
以8数码问题为例，假设初始状态为：
  

1 2 3  
4 5 6  
8 7
  
这个排列的逆序对数为5，是奇数，因此无解。无论怎样移动，逆序对数始终为奇数，无法到达目标状态：
  
1 2 3  
4 5 6  
7 8  

再举一个例子，对于4*4数码问题，假设初始状态为：

1 2 3 4  
5 6 7 8  
9 10 11 12  
13 15 14  

这个排列的逆序对数为15，是奇数，因此无解。无论怎样移动，逆序对数始终为奇数，无法到达目标状态：  

1 2 3 4  
5 6 7 8  
9 10 11 12  
13 14 15  

因此，对于n为偶数的n*n数码问题的任意一个排列，有解的条件仍然是它的逆序对数为偶数。  
  
综上，对于n*n数码问题的任意一个排列，有解的条件是它的**逆序对数为偶数**。  

### 代码实现n数码问题

题目信息：https://leetcode.cn/problems/sliding-puzzle/

In [ ]:
题目分析：
这道题不仅需要判断是否有解，还需要找到最短的移动次数


## 判断是否有解： 
逆序对的数量是否为偶数，而对于通用的 N∗N 数码问题，判定有解的一个充要条件是：「逆序对」数量为偶数，如果不满足，必然无解，直接返回-1

## 选择最短路径： A * 算法
选择最短路的过程中我们可以使用A*算法进行剪枝。A * 算法的原理就是用一个估值函数去估计当前状态的价值。
所谓A *算法其实和强化学习有异曲同工之处。A * 算法由四个函数组成。
假设起点为s，重点为t，当前状态为X。
G（x）为从起点s到x已经花费的代价 。
H（x）为从当前状态x到重点t的**估计**代价。
那么此时此刻，对起点s到终点t的函数F（x）=G（x）+ H（x） 
 F（x）函数为从s到t的代价估计，将F（x）放到优先队列中从小到大排名，就可有效减少搜索空间

## 如何估计H（x）
H（x）是在x状态对未来的估计，H函数要求如下
* 好计算
* 当到达目标状态之后，所有位置的H（x）之和应当为最小值，最好为0
1 2 3  
4 5 6  
7 8  
* H（x）应当大于等于最优（最小）需要移动的值 H*（x）
即在x状态对x到t的估计应当大于x状态从x到t的最小解。
这里，我们选择当前位置x和x的目标位置x'的曼哈顿距离作为H（x）函数。  
目标状态
1 2 3  
4 5 6  
7 8  
当前状态
1 2 3  
4 6 5 
  7 8
H（x）函数的值，曼哈顿距离
0 0 0
0 1 1
  1 1
曼哈顿距离的和为4

在一个操作中，我们如果想交换空格和某个数字x的位置，那么交换的次数就等于两者之间的曼哈顿距离。  
诶，我们注意到曼哈顿距离是一个很好的启发式函数，所以：
State表示当前节点：
将曼哈顿距离作为H(x) 就是代码中的 self.h
记录节点从s（start）到x所花费的代价G(x)（self.g）
记录节点对从s（start）到t（terminal）的估计F（x）=G（x）+ H（x）  (self.f)
根据self.f从小到大排序（堆），从小的开始遍历，该算法就是启发式搜索。 


In [7]:
# 代码
from queue import PriorityQueue
class Solution:
    def isValid(self,board:List[List[int]]) ->int:
        myboard=[board[i][j] for i in range(2) for j in range(3)]
        # 直接序列化 逆序数对的个数
        num_inverse=sum(1 for i in range(6) 
                    for j in range(i+1, 6) 
                        if myboard[j] and myboard[i] > myboard[j])
        if num_inverse%2==1:# 如果逆序数对的个数为偶数,那么这道题无解
            return -1
        else:
            return 1
    def slidingPuzzle(self, board: List[List[int]]) -> int:
        
        

SyntaxError: unexpected EOF while parsing (796833037.py, line 11)

In [ ]:
## 代码实现
import heapq
from typing import List


class Solution:
    def slidingPuzzle(self, board: List[List[int]]) -> int:
        # 邻近点
        neighbors = {0: (1, 3), 1: (0, 2, 4), 2: (1, 5), 3: (0, 4), 4: (1, 3, 5), 5: (2, 4)}

        # 曼哈顿距离
        def ManhattanDist(p1, p2):
            return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])

        # State状态
        class State:
            def __init__(self, b=None, cost=0):
                self.board = b
                self.h = self.heuristic()
                self.g = cost
                # A*, f=h+g
                self.f = self.h + self.g
                self.hash = hash(tuple(self.board))

            def __lt__(self, other):
                return self.f < other.f

            def __eq__(self, other):
                return self.hash == other.hash or self.board == other.board

            def __hash__(self):
                return self.hash

            # 两种启发式
            def heuristic(self):
                return sum(
                    ManhattanDist((i // 3, i % 3), ((num - 1) // 3, (num - 1) % 3))
                    for i, num in enumerate(self.board) if num)

            # 下一个状态为和0的各种交换
            def successor(self):
                idx = self.board.index(0)
                successors = []
                for ng in neighbors[idx]:
                    temp = self.board[:]
                    temp[idx] = temp[ng]
                    temp[ng] = 0
                    successors.append(State(temp, self.g + 1))
                return successors

        board = board[0] + board[1]
        # n*m puzzle中，m为奇数时,逆序数对奇偶性要和最终奇偶性相同，否则无解
        if sum(1 for i in range(6) for j in range(i+1, 6) if board[j] and board[i] > board[j]) % 2 == 1:
            return -1
        initState = State(board)
        pq = [initState]
        explored = {initState}
        # 优先队列以f=h+g进行了排序
        while pq:
            state = heapq.heappop(pq)
            if state.board == [1, 2, 3, 4, 5, 0]:
                return state.g
            for successor in state.successor():
                if successor not in explored:
                    explored.add(successor)
                    heapq.heappush(pq, successor)
        return -1
solution=Solution()
board = [[1,2,3],[4,0,5]]
print(solution.slidingPuzzle(board))
board = [[1,2,3],[5,4,0]]
print(solution.slidingPuzzle(board))
board = [[4,1,2],[5,0,3]]
print(solution.slidingPuzzle(board))



In [17]:
from typing import List
def isValid(board:List[List[int]]) ->int:
    # 计算逆序数对
    myboard=[board[i][j] for i in range(2) for j in range(3)]
    # 直接序列化
    inverpair=sum(1 for i in range(6) 
                for j in range(i+1, 6) 
                    if myboard[j] and myboard[i] > myboard[j]) 
    #计算逆序数对
    if inverpair% 2 == 1:# 计算逆序数对
        return -1
    else:
        return 1
board = [[1,2,3],[4,0,5]]
board = [[1,2,3],[5,4,0]]
board = [[4,1,2],[5,0,3]]
print("本次6数码问题是否有解"，isValid(board))


1
